In [133]:
from MMRcall import MMRapi
import vinwhiz as wiz
import pandas as pd
from Cnx import Cnx
from dateutil import parser

def slice(i, end, start=0):
    """
    Helper function for string manipulation.
    Takes in an object I and returns a string,
    sliced from index in I from start (default=0)
    to end.
    """
    return str(i)[start:end]

def main(vin, region, date):
    """
    Main functional interface for VINPUT.
    Input: VIN Number, State (abr).
    Output: Dataframe
    """
    # fix date format for internal use
    date = parser.parse(date)
    # Create Vehicle Descriptions Dict
    mmr = MMRapi(vin)
    mmr.match()
    mmr_make = mmr.make()
    mmr_model = mmr.model()
    mmr_style = mmr.body()
    mmr_yr = mmr.model_year()
    vehicle_desc = {
        'Make': mmr_make,
        'Model': mmr_model,
        'Style': mmr_style
    }

    algcode = wiz.vin_alg(vin)
    make_number = slice(algcode, 3)
    model_num = slice(algcode, 6, 3)
    style_num = slice(algcode, 9, 6)
    model_year = wiz.vin_year(vin)
    conn = Cnx()
    quer = f"""
        SELECT 
        [ModelYear], 
        [MakeNumber],
        [ModelNumber],
        [Style],
        [Month24],
        [Month30],
        [Month36],
        [Month42],
        [Month48],
        [Month54],
        [Month60],
        [MRM],
        [EffectiveDate],
        [Region]
        FROM [vinput].[dbo].[ALGResidualNewTable20220708]
        WHERE [MakeNumber] = {make_number} AND
        [ModelNumber] = {model_num} AND
        [Style] = {style_num} AND
        [ModelYear] = {model_year} AND
        [Region] = '{region}'
    """
    conn.set_query(quer)
    make_match = conn.execute()
    df = pd.DataFrame(make_match)
    df.columns = [
        'Model Year',
        'Make Code',
        'Model Code',
        'Style Code',
        'Month 24',
        'Month 30',
        'Month 36',
        'Month 42',
        'Month 48',
        'Month 54',
        'Month 60',
        'MRM',
        'Effective Date',
        'Region'
    ]
    
    dtmdt = lambda y: parser.parse(y)
    df['Effective Date'] = df['Effective Date'].apply(dtmdt)
    df = df.drop(df[df['Effective Date'] > date].index)
    df = df[df['Effective Date'] == df['Effective Date'].max()]
    return df.iloc[0].to_dict()
    

main('WP0AA2A92NS205471', 'S', '2022-03-06')

{'Model Year': 2022,
 'Make Code': 680,
 'Model Code': 133,
 'Style Code': 110,
 'Month 24': 52,
 'Month 30': 50,
 'Month 36': 48,
 'Month 42': 46,
 'Month 48': 43,
 'Month 54': 36,
 'Month 60': 29,
 'MRM': 121200.0,
 'Effective Date': Timestamp('2022-03-02 00:00:00'),
 'Region': 'S'}

In [100]:
from datetime import datetime
from dateutil import parser

dt_str = '2022-01-27 00:00:00.0000000'
dg_str = '2022-02-27 00:00:00.0000000'
df_str = '2022-03-27 00:00:00.0000000'
ds_str = '2022-04-27 00:00:00.0000000'

#dt_obj = datetime.strptime(dt_str, '%d-%m-%y %H:%M:%S')

#print("The type of the date is now",  type(dt_obj))
#print("The date is", dt_obj)

dt = parser.parse(dt_str)
dg = parser.parse(dg_str) 
df = parser.parse(df_str)
ds = parser.parse(ds_str)

df = pd.DataFrame({'Dates':[dt, dg, df, ds]})
dg > df

,Dates
0,True
1,False
2,False
3,False
